# Design colorswap adaptors to rescue experiment

In [1]:
%run "E:\Users\puzheng\Documents\Startup_py3.py"
sys.path.append(r"E:\Users\puzheng\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

11680


In [2]:
data_folder = r'\\storm6-pc\STORM6-FLASHDrive\Pu\20201205-proB_Dox_IAA_STI+_CTP-08'
position_file = os.path.join(data_folder, 'positions.txt')

old_positions = np.loadtxt(position_file, delimiter=',')

In [7]:
new_position_file = position_file.replace('.txt', '_adjusted.txt')

manual_shift = np.array([4.0,  -0.3])
new_positions = old_positions + manual_shift

In [8]:
np.savetxt(new_position_file, new_positions, fmt='%.2f', delimiter=',')

# Design some colorswap adaptors to finish the experiment quickly

In [9]:
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

In [15]:
# Load readouts
readout_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Readouts'
ref_files = [_fl for _fl in os.listdir(readout_folder) if 'designed_readouts' in _fl]

readout_dict = {}
for _fl in ref_files:
    _channel = int(_fl.split('designed_readouts_')[1].split('.fasta')[0])
    with open(os.path.join(readout_folder, _fl), 'r') as _rd_handle:
        for _readout in SeqIO.parse(_rd_handle, "fasta"):
            readout_dict[_readout.id] = _readout

In [16]:
readout_dict

{'Stv_91': SeqRecord(seq=Seq('TTCCCGCCAATGACGTCGGTTTGGACGAGA', SingleLetterAlphabet()), id='Stv_91', name='Stv_91', description='Stv_91 old_barcode', dbxrefs=[]),
 'Stv_92': SeqRecord(seq=Seq('GCGTTGATGTCCCTTGTGAGCGCCCGACAT', SingleLetterAlphabet()), id='Stv_92', name='Stv_92', description='Stv_92 old_barcode', dbxrefs=[]),
 'Stv_94': SeqRecord(seq=Seq('CGCTTATCGATGTCAGGTCCGCATGGGTCG', SingleLetterAlphabet()), id='Stv_94', name='Stv_94', description='Stv_94 old_barcode', dbxrefs=[]),
 'Stv_95': SeqRecord(seq=Seq('AACGTCATCGGTGGATCCCAGAGTGCCAAA', SingleLetterAlphabet()), id='Stv_95', name='Stv_95', description='Stv_95 old_barcode', dbxrefs=[]),
 'Stv_99': SeqRecord(seq=Seq('GTCCGATGAAACGTCCCGTGTGCTGTCGCG', SingleLetterAlphabet()), id='Stv_99', name='Stv_99', description='Stv_99 old_barcode', dbxrefs=[]),
 'Stv_100': SeqRecord(seq=Seq('GACACGACGTCCAAAGTTGGTCTCGCGCAA', SingleLetterAlphabet()), id='Stv_100', name='Stv_100', description='Stv_100 old_barcode', dbxrefs=[]),
 'Stv_101': SeqRec

In [12]:
# load readout sites
adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors'
readout_site_file = os.path.join(adaptor_folder, 'Readout_sites.fasta')
readout_sites = []
with open(readout_site_file, 'r') as _rd_handle:
    for _readout in SeqIO.parse(_rd_handle, "fasta"):
        readout_sites.append(_readout)
print(readout_sites)

[SeqRecord(seq=Seq('TTTGCACTGCCGTCCTTGAC', SingleLetterAlphabet()), id='Stv_82', name='Stv_82', description='Stv_82 cy7 rev-com_last20', dbxrefs=[]), SeqRecord(seq=Seq('GATCCGATTGGAACCGTCCC', SingleLetterAlphabet()), id='Stv_1', name='Stv_1', description='Stv_1 cy5 rev-com_last20', dbxrefs=[]), SeqRecord(seq=Seq('TGCGAACTGTCCGGCTTTCA', SingleLetterAlphabet()), id='Stv_79', name='Stv_79', description='Stv_79 cy3 rev-com_last20', dbxrefs=[])]


In [17]:
cy7_names = ['NDB_20', 'NDB_29']
cy3_names = ['NDB_308', 'NDB_311', 'NDB_26']

In [20]:
cy7_readouts = [_v for _k,_v in readout_dict.items() if _k in cy7_names]
cy3_readouts = [_v for _k,_v in readout_dict.items() if _k in cy3_names]

In [24]:
# generate adaptor
reload(library_tools.readouts)
from ImageAnalysis3.library_tools.readouts import Generate_adaptors

cy7_adaptors = Generate_adaptors(cy7_readouts, [readout_sites[0]])
cy3_adaptors = Generate_adaptors(cy3_readouts, [readout_sites[-1]])

In [25]:
save_folder = r'Y:\20201205-proB_Dox_IAA_STI+_CTP-08\Analysis'

In [30]:
# generate csv file to order in IDT
import csv

with open(os.path.join(save_folder, 'adjusted_probes.csv'), 'w') as csvfile:
    csvwriter = csv.writer(csvfile, #delimiter=',', 
                       quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header
    _header = ['Name', 'Sequence', 'Scale', 'Purification']
    csvwriter.writerow(_header)
    # write sequence
    for _adaptor in cy7_adaptors:
        _info = [_adaptor.id, str(_adaptor.seq), '25nm', 'STD']
        csvwriter.writerow(_info)
    for _adaptor in cy3_adaptors:
        _info = [_adaptor.id, str(_adaptor.seq), '25nm', 'STD']
        csvwriter.writerow(_info)